# Intuitive Thinker 

Improve small size LLM reasoning by employs system of thinking mental models by structured Chain-of-Thoughts process and thoughtful reflection prior to responding to user queries.

***Installation***

## A. Using Huggingface

In [2]:
from intuitive_thinker.mental_model import MentalModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import json

question="count number of r in word strawberry?"

## format question using mental model template
mental_model = MentalModel(MentalModel.CHAIN_OF_THOUGHTS)
prompt = json.loads(mental_model(question))

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
)
        
# Prepare the input as before
messages = [
    {"role": "system", "content": prompt['system_message']},
    {"role": "user", "content": prompt['user_input'] }
]

hf_model_id = "mychen76/Llama-3.1_Intuitive-Thinker"

# 1: Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(hf_model_id, device_map="auto", quantization_config=bnb_config, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(hf_model_id)

# 2: Apply the chat template
formatted_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#print("Formatted chat:\n", formatted_chat)

# 3: Tokenize the chat (This can be combined with the previous step using tokenize=True)
inputs = tokenizer(formatted_chat, return_tensors="pt", add_special_tokens=False)
# Move the tokenized inputs to the same device the model is on (GPU/CPU)
inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}
#print("Tokenized inputs:\n", inputs)

# 4: Generate text from the model
outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.1)
#print("Generated tokens:\n", outputs)

# 5: Decode the output back to a string
decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
print("Decoded output:\n", decoded_output)

Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


...use thinking_model(chain_of_thoughts,count number of r in word strawberry?)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Decoded output:
 <thinking>
a. Briefly analyze the question and outline our approach:
The question asks for the count of the letter 'r' in the word'strawberry'. This is a simple text analysis task that requires examining the word for the target letter.

b. Present a clear plan of steps to solve the problem:
1. Write out the word'strawberry' in its full form.
2. Count the occurrences of the letter 'r' in the word.
3. Provide the final count as the answer.

c. Use a "Chain of Thought" reasoning process:
1. Start by writing out the word: s-t-r-a-w-b-e-r-r-y
2. Now, count the 'r's: r (1), r (2), r (3)
3. We have found all 'r's in the word.

<reflection>
a. Review our reasoning:
We have broken down the task into simple steps and followed through with the counting process.

b. Check for potential errors or oversights:
- We have accounted for all letters in the word.
- We have specifically targeted the letter 'r' for counting.
- We have counted each occurrence of 'r' correctly.

c. Confirm ou

## Using OLLAMA API

### Ollama-Thinker

In [6]:
from intuitive_thinker.ollama_thinker import OllamaIntuitiveThinker
from intuitive_thinker.mental_model import MentalModel
import os

In [7]:
### Environment Config

ollama_url = os.getenv("OLLAMA_URL")
ollama_key = os.getenv("OLLAMA_KEY")
ollama_model = os.getenv("OLLAMA_MODEL_BASE")

print(10*"-----")
print(ollama_url, ollama_key, ollama_model)
print(10*"-----")

--------------------------------------------------
http://192.168.0.47:11434 ollama mychen76/llama3.1-intuitive-thinker:q5
--------------------------------------------------


In [8]:
query = "count number of letter r in word strawberry?"

thinker = OllamaIntuitiveThinker(llm_url=ollama_url)
response = await thinker.llmchat(question=query,
                           llm_model=ollama_model,
                           mental_model=MentalModel.CHAIN_OF_THOUGHTS)
print(10*"===")
print(response)

final_answer = thinker.parse_output(response)
print("========Final Answer=========")
print(final_answer)

...use thinking_model(chain_of_thoughts,count number of letter r in word strawberry?)



***The model didn't use the function. Its response was:

<thinking>
The task is to count the number of letter 'r' in the word "strawberry".

1. Analyze the problem:
   - The word given is "strawberry".
   - We need to count how many times the letter 'r' appears in this word.
   
2. Outline the approach:
   - Break down the word into its constituent letters.
   - Identify and count the instances of the letter 'r'.
   
3. Plan of steps:
   a. Write out the word "strawberry" with its individual letters: s-t-r-a-w-b-e-r-r-y
   b. Count the number of 'r's in the word.
   
4. Chain of Thought:
   1. The word has the following letters: s, t, r, a, w, b, e, r, r, y
   2. Identify each occurrence of 'r': there are two distinct 'r' instances.
   3. Count these occurrences to find the total number of 'r's in the word.

<reflection>
- Review: The task is to count the number of letter 'r' in "strawberry".
- Check for errors:
  - Ensure we're only counting the letter 'r', not other similar-looking letters.
  - Double-check that there are no additional 'r'

### Dynamic Model Loading 
(required ollama function calling support)

use `iceberg_thinking` to perform thinking to solve the query

#### Icerberg Mental Model

In [10]:
##
## Use: Icerberg Mental Model
##

query = """
apply iceberg_thinking thinking for query:
Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?]
"""

thinker = OllamaIntuitiveThinker(llm_url=ollama_url)
response = await thinker.llmchat(question=query,
                           llm_model=ollama_model,
                           mental_model=None)
print(10*"===")
print(response)

final_answer = thinker.parse_output(response)
print("========Final Answer=========")
print(final_answer)

...calling tool:

[
    {
        'function': {
            'name': 'apply_thinking_model',
            'arguments': {
                'model': 'iceberg_thinking',
                'query': "Patel family who is planning a road trip from Toronto to Vancouver, covering a total 
distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km
longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to 
complete the trip in 5 days?"
            }
        }
    }
]

...use thinking_model(iceberg_thinking,Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?)



***after tool call response***

<thinking>
To solve this problem, we need to find the value of x that will allow the Patel family to complete their road trip in 5 days. 

1. First, let's identify the total distance they need to cover (4,400 km).
2. Next, we know each day's drive is 15 km longer than the previous day.
3. We can use algebraic expressions to represent this situation:
   Let x be the distance driven on the first day.
   Then, the distances for the next four days are: x + 15, x + 30, x + 45, and x + 60.
4. The sum of these distances should equal the total distance (4,400 km).
5. We can write this as an equation:

x + (x + 15) + (x + 30) + (x + 45) + (x + 60) = 4400

6. Combine like terms:
   5x + 150 = 4400
7. Subtract 150 from both sides:
   5x = 4250
8. Divide by 5 to solve for x:
   x = 850

Therefore, the Patel family should drive 850 km on the first day.

<reflection>
Reviewing our reasoning:
- We correctly identified the total distance and the increase in distance each day.
- Our algebraic expressio

#### Thinking Fast and Slow 

In [9]:
##
## Use: Thinking Fast and Slow 
##
query = """
apply thinking_fast_and_slow thinking for query:
Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?]
"""

thinker = OllamaIntuitiveThinker(llm_url=ollama_url)
response = await thinker.llmchat(question=query,
                           llm_model=ollama_model,
                           mental_model=None)
print(10*"===")
print(response)

final_answer = thinker.parse_output(response)
print("========Final Answer=========")
print(final_answer)

...calling tool:

[
    {
        'function': {
            'name': 'apply_thinking_model',
            'arguments': {
                'model': 'thinking_fast_and_slow',
                'query': "A family plans a 5-day road trip from Toronto to Vancouver. They want equal daily 
distances, increasing by 15 km each day. What's the first day's distance?"
            }
        }
    }
]

...use thinking_model(thinking_fast_and_slow,A family plans a 5-day road trip from Toronto to Vancouver. They want equal daily distances, increasing by 15 km each day. What's the first day's distance?)



***after tool call response***

<thinking>
<reflection>

1. The total distance of the trip is 4,400 km.
2. The family wants to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day.
3. Let x be the distance driven on the first day.

Chain of Thought:
Step 1: Calculate the total increase in distance over 5 days
Total increase = 15 km/day * 5 days = <<15*5=75>>75 km

Step 2: Set up an equation for the total distance
x + (x+15) + (x+30) + (x+45) + (x+60) = 4400

Step 3: Simplify and solve the equation
5x + 150 = 4400
5x = 4250
x = 850

<reflection>
Is the distance increasing by 15 km each day?
Yes, it is.

Does the total increase in distance over 5 days make sense?
Yes, it does (75 km).

Is my calculation for x correct?
Yes, I confirmed with a step-by-step solution.

Conclusion:
The family should drive 850 km on the first day to complete the trip in 5 days.

<output>
The value of x that will allow the Patel family to complete their road trip from Toronto to Vancouv

#### Second Order Thinking 

In [10]:
##
## Use: Second Order Thinking 
##
query = """
apply second_order_thinking thinking for query:
Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?]
"""

thinker = OllamaIntuitiveThinker(llm_url=ollama_url)
response = await thinker.llmchat(question=query,
                           llm_model=ollama_model,
                           mental_model=None)
print(10*"===")
print(response)

final_answer = thinker.parse_output(response)
print("========Final Answer=========")
print(final_answer)

...calling tool:

[
    {
        'function': {
            'name': 'apply_thinking_model',
            'arguments': {
                'model': 'second_order_thinking',
                'query': "Patel family who is planning a road trip from Toronto to Vancouver, covering a total 
distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km
longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to 
complete the trip in 5 days?}"
            }
        }
    }
]

...use thinking_model(second_order_thinking,Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?})



***after tool call response***

<thinking>
The Patel family needs to cover a total distance of 4,400 km in 5 days, with each day's drive being 15 km longer than the previous day. We need to find the value of x that will allow them to complete the trip in 5 days.

Let's break down the problem:

1. The total distance is 4,400 km.
2. The family will drive for 5 days.
3. Each day's drive is 15 km longer than the previous day.
4. We need to find the value of x that will allow them to complete the trip in 5 days.

We can start by finding the total distance driven on each day:

Day 1: x km
Day 2: x + 15 km
Day 3: x + 30 km
Day 4: x + 45 km
Day 5: x + 60 km

The sum of the distances driven on each day is equal to the total distance:

x + (x + 15) + (x + 30) + (x + 45) + (x + 60) = 4,400

Combine like terms:

5x + 150 = 4,400

Subtract 150 from both sides:

5x = 4,250

Divide by 5:

x = 850

Therefore, the value of x is 850.

<reflection>
Let's review our reasoning and check for potential errors or oversights.

1. We correctl

### Non-Linear Thinking

In [11]:
##
## Use: Non-Linear Thinking
##
query = """
apply nonlinear_thinking thinking for query:
Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?]
"""

thinker = OllamaIntuitiveThinker(llm_url=ollama_url)
response = await thinker.llmchat(question=query,
                           llm_model=ollama_model,
                           mental_model=None)
print(10*"===")
print(response)

final_answer = thinker.parse_output(response)
print("========Final Answer=========")
print(final_answer)

...calling tool:

[
    {
        'function': {
            'name': 'apply_thinking_model',
            'arguments': {
                'model': 'nonlinear_thinking',
                'query': "Patel family who is planning a road trip from Toronto to Vancouver, covering a total 
distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km
longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to 
complete the trip in 5 days?"
            }
        }
    }
]

...use thinking_model(nonlinear_thinking,Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?)



***after tool call response***

<thinking>
To solve this problem, we need to find the value of x that will allow the Patel family to complete their road trip in 5 days. We know that each day's drive is 15 km longer than the previous day, and they want to divide the total distance of 4,400 km into equal parts over 5 days.

Here's a step-by-step approach to solve this problem:

1. Let x be the distance driven on the first day.
2. The distance driven on each subsequent day will be:
   Day 2: x + 15
   Day 3: (x + 15) + 15 = x + 30
   Day 4: (x + 30) + 15 = x + 45
   Day 5: (x + 45) + 15 = x + 60

3. The total distance driven over 5 days is the sum of the distances driven each day:
   Total Distance = x + (x + 15) + (x + 30) + (x + 45) + (x + 60)

4. We know that the total distance is 4,400 km, so we can set up an equation and solve for x:

   x + (x + 15) + (x + 30) + (x + 45) + (x + 60) = 4400

5. Combine like terms and simplify the equation:
   5x + 150 = 4400
   5x = 4250
   x = 850

Therefore, the Patel family shoul

### Critical Thinking

In [12]:
##
## Use: Critical Thinking
##
query = """
apply critical_thinking thinking for query:
Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?]
"""

thinker = OllamaIntuitiveThinker(llm_url=ollama_url)
response = await thinker.llmchat(question=query,
                           llm_model=ollama_model,
                           mental_model=None)
print(10*"===")
print(response)

final_answer = thinker.parse_output(response)
print("========Final Answer=========")
print(final_answer)

...calling tool:

[
    {
        'function': {
            'name': 'apply_thinking_model',
            'arguments': {
                'model': 'critical_thinking',
                'query': "Patel family who is planning a road trip from Toronto to Vancouver, covering a total 
distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km
longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to 
complete the trip in 5 days?"
            }
        }
    }
]

...use thinking_model(critical_thinking,Patel family who is planning a road trip from Toronto to Vancouver, covering a total distance of 4,400 km. They want to divide the drive into equal parts over 5 days, with each day's drive being 15 km longer than the previous day. If they drive x km on the first day, what is the value of x that will allow them to complete the trip in 5 days?)



***after tool call response***

<critical-thinking>
1. Analyze the problem:
   - The Patel family needs to cover a total distance of 4,400 km in 5 days.
   - Each day's drive must be 15 km longer than the previous day.

2. Outline the approach:
   - Calculate the number of kilometers driven each day.
   - Ensure that the total distance covered over 5 days equals 4,400 km.
   - Use algebra to represent the unknown value of x and solve for it.

3. Chain of Thought:

Step 1: Let's define the distance driven on each day as follows:
- Day 1: x km
- Day 2: x + 15 km
- Day 3: x + 30 km
- Day 4: x + 45 km
- Day 5: x + 60 km

Step 2: The total distance covered over the 5 days should equal 4,400 km. So, we can set up an equation:

x + (x + 15) + (x + 30) + (x + 45) + (x + 60) = 4400

Combine like terms and simplify the equation.

4. Solve for x:
   - 5x + 150 = 4400
   - 5x = 4400 - 150
   - 5x = 4250
   - x = 4250 / 5
   - x = 850

Step 5: Now that we have found the value of x, let's reflect on our process.

<reflection>
a. D

### Llama-Thinker

In [21]:
### Llama-Thinker

from intuitive_thinker.llama_thinker import LlamaIntuitiveThinker
from intuitive_thinker.mental_model import MentalModel
import os
from dotenv import load_dotenv
load_dotenv()

### Environment Config
openai_url = os.getenv("OPENAI_URL")
openai_api_key = os.getenv("OPENAI_KEY")
openai_model = os.getenv("OPENAI_MODEL")

print(10*"-----")
print(openai_url, openai_api_key, openai_model)
print(10*"-----")

--------------------------------------------------
http://192.168.0.26:11434/v1/ ollama mychen76/llama3.1-intuitive-thinker:q5
--------------------------------------------------


In [23]:
query = "count number of letter r in word strawberry?"

openthinker = LlamaIntuitiveThinker(llm_url=openai_url, llm_api_key=openai_api_key)
response = openthinker.llmchat(question=query, llm_model=openai_model, 
                               use_streaming=False,
                               mental_model=MentalModel.CHAIN_OF_THOUGHTS,
                              options={"temperature":1.0,"max_tokens":1024})
## {"temperature":1.5,"max_tokens":512}
print("======Full Response=========")
print(response)
print("======Final Answer=========")
answer= openthinker.parse_output(response)
print(answer)

...use thinking_model(chain_of_thoughts,count number of letter r in word strawberry?)

======Full Response=========
<thinking>

To calculate the number of letter R's within word “strawberry”.

1. Step through how we'll determine quantity.
The plan includes breaking down and reading term.
Looking by examining spelling and identify
any duplicates in process thus to know our answer in conclusion

There are seven items in what I've outlined:
      -step-by-anagram-of-and-name,
      -exam break and read word as individual letters out so spelling be soundly understood.
       Breaks then look for duplicate r.
      Count R.
      Identify number of
   Duplicates if one found and report on finding


2. Consider step to check if each might have errors or is the way the problem might better still approach.
   So step need reflection.

What about reading spelling forward instead going left word? Which letter comes from each spot.



       Checking in case there might an answer easier via spell